## 实验用，简单均线策略

In [4]:
import pymongo
import pandas as pd
import numpy as np

In [5]:
start = "20210601"
symbol_list = ['SZSE.000301', 'SZSE.300316', 'SZSE.002739', 'SZSE.002064', 'SZSE.002459']
initial_fund = 100000
freq = 'day'

In [6]:
connect = pymongo.MongoClient(host='192.168.214.199', port=27017)
db = connect['lsqt_kline']
df = pd.DataFrame(list(db['stock_day_1'].find({"$and":[{"symbol": {"$regex": "^SZSE", "$in":symbol_list}}, {"trade_day":{"$gte": start}}] })))
df

,_id,trade_day,open,close,high,low,volume,turn_volume,swing,change_ratio,change,turn_over,symbol,source,type,datetime
0,619ca8750573f7486c93fa5f,20210601,31.15,31.22,31.49,30.06,233161,3.988353e+08,4.59,0.22,0.07,1.35,SZSE.000301,eastmoney,day,2021-06-01
1,619ca8750573f7486c93fa60,20210602,31.40,31.44,31.56,30.59,190537,3.310064e+08,3.11,0.70,0.22,1.10,SZSE.000301,eastmoney,day,2021-06-02
2,619ca8750573f7486c93fa61,20210603,31.25,30.16,31.42,30.10,292135,4.982390e+08,4.20,-4.07,-1.28,1.69,SZSE.000301,eastmoney,day,2021-06-03
3,619ca8750573f7486c93fa62,20210604,30.15,30.64,30.84,29.87,152142,2.570069e+08,3.22,1.59,0.48,0.88,SZSE.000301,eastmoney,day,2021-06-04
4,619ca8750573f7486c93fa63,20210607,30.20,30.66,30.91,29.94,152916,2.581242e+08,3.17,0.07,0.02,0.88,SZSE.000301,eastmoney,day,2021-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,619cae890573f7486ce3d868,20211117,641.41,640.81,650.68,630.94,113984,8.418722e+08,3.07,-0.20,-1.29,0.94,SZSE.300316,eastmoney,day,2021-11-17
586,619cae890573f7486ce3d869,20211118,639.87,619.88,639.87,609.06,170163,1.220462e+09,4.81,-3.27,-20.93,1.41,SZSE.300316,eastmoney,day,2021-11-18
587,619cae890573f7486ce3d86a,20211119,619.70,623.99,631.29,604.69,180188,1.282895e+09,4.29,0.66,4.11,1.49,SZSE.300316,eastmoney,day,2021-11-19
588,619cae890573f7486ce3d86b,20211122,624.59,662.09,665.95,619.36,202982,1.514346e+09,7.47,6.11,38.10,1.68,SZSE.300316,eastmoney,day,2021-11-22


In [7]:
hold_list = []
deal_record = []
current_fund = initial_fund
def buy(symbol, price, date):
    global current_fund
    current_fund -= price*100
    hold_list.append(symbol)
    deal_record.append("{} 买入 {} 共 100 手".format(date, symbol))
    
def sell(symbol, price, date):
    global current_fund
    current_fund += price*100
    hold_list.remove(symbol)
    deal_record.append("{} 卖出 {} 共 100 手".format(date, symbol))


for symbol in symbol_list:
    temp_df = df[df['symbol']==symbol]
    i = 20
    while i<len(temp_df):
        ma5 = temp_df['close'][i-5:i].mean()
        ma20 = temp_df['close'][i-20:i].mean()
        if ma5>ma20 and symbol not in hold_list:
            buy(symbol, temp_df['close'].iloc[i],temp_df['trade_day'].iloc[i])
        if ma5<ma20 and symbol in hold_list:
            sell(symbol, temp_df['close'].iloc[i],temp_df['trade_day'].iloc[i])
        i+=1
print(deal_record)
print(current_fund)
print('收益率为{}%'.format((current_fund-initial_fund)/initial_fund*100))

['20210630 买入 SZSE.000301 共 100 手', '20210809 卖出 SZSE.000301 共 100 手', '20210817 买入 SZSE.000301 共 100 手', '20210927 卖出 SZSE.000301 共 100 手', '20211026 买入 SZSE.000301 共 100 手', '20211104 卖出 SZSE.000301 共 100 手', '20210630 买入 SZSE.300316 共 100 手', '20210819 卖出 SZSE.300316 共 100 手', '20210827 买入 SZSE.300316 共 100 手', '20210922 卖出 SZSE.300316 共 100 手', '20211019 买入 SZSE.300316 共 100 手', '20211119 卖出 SZSE.300316 共 100 手', '20210706 买入 SZSE.002739 共 100 手', '20210721 卖出 SZSE.002739 共 100 手', '20210820 买入 SZSE.002739 共 100 手', '20210930 卖出 SZSE.002739 共 100 手', '20211019 买入 SZSE.002739 共 100 手', '20211027 卖出 SZSE.002739 共 100 手', '20211115 买入 SZSE.002739 共 100 手', '20210630 买入 SZSE.002064 共 100 手', '20210726 卖出 SZSE.002064 共 100 手', '20210817 买入 SZSE.002064 共 100 手', '20210823 卖出 SZSE.002064 共 100 手', '20210831 买入 SZSE.002064 共 100 手', '20210927 卖出 SZSE.002064 共 100 手', '20211027 买入 SZSE.002064 共 100 手', '20211029 卖出 SZSE.002064 共 100 手', '20210630 买入 SZSE.002459 共 100 手', '20210819 卖出 SZSE.0